# FA691 Homework 2
# Due: Wednesday, February 1 @ 11:59PM

Name: 

Date: 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set seed of random number generator
CWID = -1 #Place here your Campus wide ID number, this will personalize
#your results, but still maintain the reproduceable nature of using seeds.
#If you ever need to reset the seed in this assignment, use this as your seed
#Papers that use -1 as this CWID variable will earn 0's so make sure you change
#this value before you submit your work.
personal = CWID % 10000
np.random.seed(personal)

## Question 1 (15pt)

### Question 1.1
Use the `yfinance` package (or other method of your choice) to obtain the daily adjusted close prices for 2 different stocks.  You should have at least two years of data for both assets.  You should inspect the dates for your data to make sure you are including everything appropriately.  Create a data frame (or array) of the daily log returns both both stocks along with the lagged returns (2 lags).  Use the `print` command to display your data.

In [2]:
# Enter your code here

### Question 1.2
Split your data into training and testing sets (80% training and 20% test).

Linearly regress one of your stock returns as a function of the lagged returns (2 lags) for both stocks.
This should be of the form $r_{1,t} = \beta_0 + \beta_{1,1} r_{1,t-1} + \beta_{1,2} r_{1,t-2} + \beta_{2,1} r_{2,t-1} + \beta_{2,2} r_{2,t-2}$.
Evaluate the performance of this model with the mean squared error on the test data.

In [3]:
# Enter your code here

<font color='red'>Solution:</font>

## Question 2 (35pt)

### Question 2.1
Using the same data, train/test split ratio, and consider the same regression problem as in Question 1.2.
Create a feed-forward neural network with a single hidden layer (2 hidden nodes) densely connected to the inputs.
You may choose any activation functions you wish.

In [4]:
# Enter your code here

### Question 2.2
Train this neural network on the training data.  
Evaluate the performance of this model with the mean squared error on the test data.

In [5]:
# Enter your code here

<font color='red'>Solution:</font> 

### Question 2.3
Using the same train/test split and consider the same regression problem as in Question 1.2.
Train and test another neural network of your own design.

In [6]:
# Enter your code here

<font color='red'>Solution:</font>

## Question 3 (10pt)

## Question 3.1
Consider the same regression problem as in Question 1.2.
Of the methods considered in this assignment, which would you recommend in practice?
Explain briefly (1 paragraph) why you choose this fit. 

<font color='red'>Solution:</font> 